In [52]:
import requests
import pandas as pd
from pathlib import Path

In [53]:
# Путь к файлу
filepath  = Path('/content/drive/MyDrive/df_dota2_matches.csv')

In [54]:
# Загрузка файла
df_old = pd.read_csv(filepath)

In [55]:
my_list = df_old['match_id'].tolist()

In [56]:
match_id_for_api = []

def check_match_id(match_list, df):
    for match_id in match_list:
        if match_id in df['match_id'].values:
            continue
        match_id_for_api.append(match_id)
        if len(match_id_for_api) == 60:
            break
    return match_id_for_api

def get_match_data(match_id):
    url = f'https://api.opendota.com/api/matches/{match_id}'
    response = requests.get(url)
    data = response.json()
    df = pd.json_normalize(data)
    for column in df.columns:
        if isinstance(df[column][0], dict):
            nested_df = pd.json_normalize(df[column][0])
            nested_df.columns = [f"{column}.{col}" for col in nested_df.columns]
            df = pd.concat([df, nested_df], axis=1)
            df.drop([column], axis=1, inplace=True)
    return df

def players_to_dataframe(df):
    result = pd.DataFrame()
    for i, row in df.iterrows():
        match_id = row['match_id']
        players = row['players']
        player_df = pd.DataFrame(players)
        player_df['match_id'] = match_id
        result = pd.concat([result, player_df])
    return result

In [ ]:
check_match_id(my_list, y)

In [ ]:
match_id_for_api.len()

In [67]:
data = pd.concat([get_match_data(match_id) for match_id in match_id_for_api])

In [69]:
data

,match_id,barracks_status_dire,barracks_status_radiant,chat,cluster,dire_score,dire_team_id,draft_timings,duration,engine,...,cosmetics.8552,cosmetics.8558,cosmetics.8560,cosmetics.14488,cosmetics.18033,cosmetics.18398,cosmetics.26372,cosmetics.26423,all_word_counts.непобедимый,all_word_counts.паш
0,7.124356e+09,63.0,51.0,"[{'time': -253, 'type': 'chat', 'key': 'ojo x ...",121.0,5.0,None,"[{'order': 1, 'pick': False, 'active_team': 2,...",607.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,7.132443e+09,63.0,48.0,None,152.0,46.0,None,None,1526.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,7.132456e+09,0.0,63.0,None,145.0,12.0,None,None,1416.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,7.132455e+09,63.0,3.0,"[{'time': -24, 'type': 'chatwheel', 'key': '71...",271.0,24.0,None,"[{'order': 1, 'pick': False, 'active_team': 2,...",1486.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,7.132451e+09,60.0,15.0,None,273.0,12.0,None,None,1599.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
x = players_to_dataframe(data)

ValueError: ignored

In [ ]:
x = x[['match_id', 'player_slot', 'account_id', 'backpack_0', 'backpack_1', 
       'backpack_2', 'gold_per_min', 'xp_per_min', 'duration', 'win', 'hero_id', 
       'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5']]

In [ ]:
frames = [y, x]
y = pd.concat(frames)
y.shape

In [70]:
y

,match_id,player_slot,account_id,backpack_0,backpack_1,backpack_2,gold_per_min,xp_per_min,duration,win,hero_id,item_0,item_1,item_2,item_3,item_4,item_5
0,7124356205,0,3.911517e+08,0,0,0,99,236,607,0,19,29,0,11,0,0,0
1,7124356205,1,1.106038e+09,0,0,0,103,122,607,0,13,16,16,16,0,237,44
2,7124356205,2,1.355229e+08,0,0,0,290,266,607,0,66,94,44,43,28,65,0
3,7124356205,3,NaN,0,0,0,112,144,607,0,128,29,0,0,0,0,0
4,7124356205,4,1.313038e+08,0,0,0,292,308,607,0,76,29,261,16,34,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,7134230905,128,NaN,75,0,11,733,546,1608,1,109,63,236,147,160,0,51
6,7134230905,129,1.640303e+08,0,0,188,427,399,1608,1,86,48,254,40,226,244,36
7,7134230905,130,NaN,0,0,0,319,342,1608,1,102,931,40,73,188,36,0
8,7134230905,131,3.743816e+08,0,0,0,656,617,1608,1,59,29,210,156,31,151,117
